In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,r2_score


Read data


In [47]:
movie_data=pd.read_csv("IMDb_Movies_India.csv",encoding='latin1')

Preprocess

In [48]:
movie_data.head()
movie_data.isnull().sum()



Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

remove null rows

remove duplictes

In [49]:
movie_data.dropna(inplace=True)

In [50]:
movie_data['Name'] = movie_data['Name'].str.extract('([A-Za-z\s\'\-]+)')
print(type(movie_data['Year']))
movie_data['Year'] = movie_data['Year'].astype(str).str.replace(r'[()]', '',regex=True).astype(int)
movie_data['Duration']=pd.to_numeric(movie_data['Duration'].astype(str).str.replace(r' min','',regex=True),errors='coerce')
# df['Duration'] = pd.to_numeric(df['Duration'].str.replace(r' min', '', regex=True), errors='coerce')
movie_data['Votes']=pd.to_numeric(movie_data['Votes'].astype(str).str.replace(',',''),errors='coerce')
movie_data.drop_duplicates(subset=['Name'],inplace=True)
movie_data.dropna(inplace=True)
movie_data.info()

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
Index: 5282 entries, 1 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      5282 non-null   object 
 1   Year      5282 non-null   int32  
 2   Duration  5282 non-null   int64  
 3   Genre     5282 non-null   object 
 4   Rating    5282 non-null   float64
 5   Votes     5282 non-null   int64  
 6   Director  5282 non-null   object 
 7   Actor 1   5282 non-null   object 
 8   Actor 2   5282 non-null   object 
 9   Actor 3   5282 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(6)
memory usage: 433.3+ KB


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Kaustubha Ram\AppData\Local\Temp\ipykernel_27820\3894014875.py:1: SyntaxWarning: invalid escape sequence '\s'
  movie_data['Name'] = movie_data['Name'].str.extract('([A-Za-z\s\'\-]+)')


Model

In [51]:
model1=RandomForestClassifier()
model2=LinearRegression()
